[Reference](https://medium.com/python-in-plain-english/accessing-the-dark-web-with-python-4db013bf4d32)

In [1]:
!pip3 install requests stem pysocks fake_useragent

     |████████████████████████████████| 2.9MB 5.2MB/s 
  Created wheel for stem: filename=stem-1.8.0-cp36-none-any.whl size=436039 sha256=8db564ec6bd53da9f844828111de78eb6782f1c34df51bb4dcc0181268b5302b
  Stored in directory: /root/.cache/pip/wheels/02/3a/ee/1094b166e029353f892c0b121aa02f48aff5e658396924bc2a
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=1013b13bdbe77a49ce4798201cf27936c3ab60a013c4c2a44f5ad158ced2df8c
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built stem fake-useragent


In [2]:
from stem import Signal
from stem.control import Controller
from requests import get
from fake_useragent import UserAgent
from time import sleep


def new_tor_id():
  with Controller.from_port(port=9051) as controller:
    controller.authenticate(password="your Tor password")
    controller.signal(Signal.NEWNYM)

  
def test_creation_of_new_identity(url: str):
    tor_proxy = {
      "http": "socks5h://127.0.0.1:9050",
      "https": "socks5h://127.0.0.1:9050"
    }
    headers = {
      "User-Agent": UserAgent().random
    }
    resp = get(url, headers=headers, proxies=tor_proxy)
    return resp
 

if __name__ == "__main__":
  for _ in range(10):
    resp = test_creation_of_new_identity("http://httpbin.org/ip")
    print(resp.text)
    new_tor_id()
    # need to wait at least 5 seconds before generating new exit IP
    sleep(5)

In [3]:
from stem import Signal
from stem.control import Controller
from argparse import ArgumentParser as AP
from fake_useragent import UserAgent
from requests import get
from os import environ
from bs4 import BeautifulSoup as bs
from colored import fg, attr
from re import compile, findall, IGNORECASE
from prettytable import PrettyTable as PT
from random import randint


row_data_cell_re = compile(r'<td [\w="]+>([\w\s\.@:()\'-0-9]+)<\/td>')


def extract_row_data_cells(row: str):
        return row_data_cell_re.findall(row, IGNORECASE)
        

def parse_response(resp: str):
    html_parser = bs(resp.text, "html.parser")

    all_tables = html_parser.findAll("table")
    general_details = all_tables[2]
    physical_appearance = all_tables[3]
    personality = all_tables[4]
    federal_taxpayer_id_nums = all_tables[5]
    
    general_details_tbl = PT()
    general_details_tbl.field_names = [
        "%sGeneral Details%s" % (fg(randint(1, 220)), attr(0)), 
        "%sValues%s" % (fg(randint(1, 220)), attr(0))
        ]
    for row in general_details.findAll("tr"):
        v = extract_row_data_cells(str(row))
        if len(v) != 2: continue
        else: general_details_tbl.add_row(v)
    print(general_details_tbl)

    physical_appearance_tbl = PT()
    physical_appearance_tbl.field_names = [
        "%sPhysical Appearance%s" % (fg(randint(1, 220)), attr(0)), 
        "%sValues%s" % (fg(randint(1, 220)), attr(0))
        ]
    for row in physical_appearance.findAll("tr"):
        v = extract_row_data_cells(str(row))
        if len(v) != 2: continue
        else: physical_appearance_tbl.add_row(v)
    print(physical_appearance_tbl)

    personality_tbl = PT()
    personality_tbl.field_names = [
        "%sPersonality Traits%s" % (fg(randint(1, 220)), attr(0)), 
        "%sValues%s" % (fg(randint(1, 220)), attr(0))
        ]
    for row in personality.findAll("tr"):
        v = extract_row_data_cells(str(row))
        if len(v) != 2: continue
        else: personality_tbl.add_row(v)
    print(personality_tbl)

    federal_taxpayer_tbl = PT()
    federal_taxpayer_tbl.field_names = [
        "%sUS Federal Taxpayer Id Number (Tin)%s" % (fg(randint(1, 220)), attr(0)), 
        "%sTax ID%s" % (fg(randint(1, 220)), attr(0))
        ]
    for row in federal_taxpayer_id_nums.findAll("tr"):
        v = extract_row_data_cells(str(row))
        if len(v) != 2: continue
        else: federal_taxpayer_tbl.add_row(v)
    print(federal_taxpayer_tbl)


def make_request(url: str, gender: str):
  tor_proxy = {
        "http": "socks5h://127.0.0.1:9050",
        "https": "socks5h://127.0.0.1:9050"
    }

    random_user_agent = UserAgent().random
    headers = {
        "User-Agent": random_user_agent
        }

    if gender == "female":
        url = url + "?gender=f"
    elif gender == "male":
        url = url + "?gender=m"
    else: 
        url = url + "?gender=r"

    return get(url, headers=headers, proxies=tor_proxy)


def create_new_tor_ip():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate(password=environ["TOR_PASS"])
        controller.signal(Signal.NEWNYM)

    
if __name__ == "__main__":
    hidden_service_url = "http://elfq2qefxx6dv3vy.onion/fakeid.php"

    parser = AP(description="Generate identity to browse the Dark Web")
    parser.add_argument(
        "-g", "--gender",
        choices=("female", "male", "random"),
        default="random",
        help="Select the gender to generate a Fake Id for"
        )
    args = parser.parse_args()

    resp = make_request(hidden_service_url, args.gender)
    parse_response(resp)